In [ ]:
import pandas as pd

from tableone import TableOne

In [ ]:
data = pd.read_gbq("""
WITH ed AS (
    SELECT
          stays.hadm_id
        , SUM(TIMESTAMP_DIFF(stays.outtime, stays.intime, HOUR)) AS ed_los
        , MAX(triage.acuity) AS ed_acuity
    FROM `lcp-internal.mimic_ed.edstays` stays
    LEFT JOIN `lcp-internal.mimic_ed.triage` triage
        ON stays.stay_id = triage.stay_id
    LEFT JOIN `lcp-internal.mimic_ed.diagnosis` diag
        ON stays.stay_id = diag.stay_id
        AND diag.seq_num = 1
    GROUP BY stays.hadm_id
)
SELECT
      pat.subject_id
    , adm.hadm_id
    , icu.stay_id
    , CASE
        WHEN FIRST_VALUE(icu.stay_id) OVER icustay_window = icu.stay_id THEN 1
        ELSE 0
      END AS pat_count
    , CASE
        WHEN FIRST_VALUE(adm.hadm_id) OVER hadm_window = adm.hadm_id THEN 1
        ELSE 0
      END AS hadm_count
    , pat.anchor_age + (EXTRACT(YEAR FROM icu.intime) - pat.anchor_year) AS age
    , pat.gender
    , adm.insurance
    , CAST(ed.ed_acuity AS INTEGER) AS ed_acuity
    , ed.ed_los
    , icu.first_careunit
    , icu.los AS icu_los
    , TIMESTAMP_DIFF(adm.dischtime, adm.admittime, HOUR) / 24 AS hosp_los
    , adm.hospital_expire_flag
FROM `lcp-internal.mimic_hosp.patients` pat
INNER JOIN `lcp-internal.mimic_hosp.admissions` adm
    ON pat.subject_id = adm.subject_id
INNER JOIN `lcp-internal.mimic_icu.icustays` icu
    ON adm.hadm_id = icu.hadm_id
LEFT JOIN ed
    ON adm.hadm_id = ed.hadm_id
WINDOW hadm_window AS (PARTITION BY pat.subject_id ORDER BY adm.admittime)
     , icustay_window AS (PARTITION BY pat.subject_id ORDER BY icu.intime)
""", "lcp-internal")

In [ ]:
int_cols = data.dtypes.values=="Int64"
data.loc[:, int_cols] = data.loc[:, int_cols].astype(float)
data.loc[:, int_cols] = data.loc[:, int_cols].astype(int, errors="ignore")

In [ ]:
columns = [
    "pat_count", "hadm_count",
    "age", "gender", "insurance",
    "first_careunit",
    "ed_acuity", "ed_los",
    "icu_los", "hosp_los"
]

categorical = [
    "pat_count", "hadm_count",
    "gender", "insurance",
    "first_careunit",
    "ed_acuity",
]

order = {
    "pat_count": [1, 0], "hadm_count": [1, 0],
    "gender": ["F", "M"]
}

limit = {
    "pat_count": 1, "hadm_count": 1,
    "gender": 1
}

rename = {
    "pat_count": "Distinct patients", "hadm_count": "Distinct hospitalizations",
    "age": "Age", "gender": "Gender", "insurance": "Insurance",
    "first_careunit": "First ICU stay, unit type",
    "ed_acuity": "ED acuity", "ed_los": "ED length of stay",
    "icu_los": "ICU length of stay", "hosp_los": "Hospital length of stay"
}

table = TableOne(data, columns=columns, categorical=categorical, order=order, limit=limit, rename=rename,
                 groupby="hospital_expire_flag")
table

## Summary figures

In [ ]:
# Table 1 Demographics of the admissions in the database.


In [ ]:
# Table 2 Most frequent diagnoses as coded using the ICD codes.
query = """
WITH grp AS
(
SELECT a.hadm_id
, CASE WHEN icu.hadm_id IS NOT NULL THEN 1 ELSE 0 END AS has_icu_admit
, CASE WHEN dia.seq_num > 1 THEN 0 ELSE 1 END AS primary_dx
, dia.icd_code
, dia.icd_version
, d_dx.long_title
FROM `lcp-internal.mimic_hosp.admissions` a
LEFT JOIN `lcp-internal.mimic_hosp.diagnoses_icd` dia ON a.hadm_id = dia.hadm_id
LEFT JOIN `lcp-internal.mimic_hosp.d_icd_diagnoses` d_dx ON dia.icd_code = d_dx.icd_code AND dia.icd_version = d_dx.icd_version
-- join to icustays and flag those which have an ICU stay
LEFT JOIN ( SELECT DISTINCT hadm_id FROM `lcp-internal.mimic_icu.icustays` ) icu ON a.hadm_id = icu.hadm_id
)
SELECT has_icu_admit, primary_dx, icd_code, icd_version, long_title
, COUNT(*) AS n
FROM grp
GROUP BY 1, 2, 3, 4, 5
"""

dx = pd.read_gbq(query, "lcp-internal")
dx.head()

In [ ]:
idx = dx['has_icu_admit'] == 1
print('Top 10 dx for stays with an ICU admission:')
grp = dx.loc[idx].groupby(['icd_code', 'long_title'])[['n']].sum().sort_values('n', ascending=False)
display(grp.head(n=10))

print('Top 10 dx for stays without an ICU admission:')
grp = dx.loc[~idx].groupby(['icd_code', 'long_title'])[['n']].sum().sort_values('n', ascending=False)
display(grp.head(n=10))

idx = (dx['has_icu_admit'] == 1) & (dx['primary_dx'] == 1)
print('Top 10 *primary* dx for stays with an ICU admission:')
grp = dx.loc[idx].groupby(['icd_code', 'long_title'])[['n']].sum().sort_values('n', ascending=False)
display(grp.head(n=10))

idx = (dx['has_icu_admit'] == 0) & (dx['primary_dx'] == 1)
print('Top 10 *primary* dx for stays without an ICU admission:')
grp = dx.loc[idx].groupby(['icd_code', 'long_title'])[['n']].sum().sort_values('n', ascending=False)
display(grp.head(n=10))

In [ ]:
# Figure 2: Visualization of a single patient's stay.

### Data completion for ICU stays

In [ ]:
# Figure 3: Data completion grouped by table.
query = """
SELECT a.hadm_id
, CASE WHEN icu.hadm_id IS NOT NULL THEN 1 ELSE 0 END AS has_icu_stay
,tra.n AS n_transfers
,dia.n AS n_diagnoses_icd
,drg.n AS n_drgcodes
,ema.n AS n_emar
,hcp.n AS n_hcpcsevents
,lab.n AS n_labevents
,mic.n AS n_microbiologyevents
,omr.n AS n_omr
,pha.n AS n_pharmacy
,poe.n AS n_poe
,pre.n AS n_prescriptions
,pro.n AS n_procedures_icd
,ser.n AS n_services
FROM `lcp-internal.mimic_hosp.admissions` a
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.transfers` GROUP BY hadm_id ) tra ON a.hadm_id = tra.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.diagnoses_icd` GROUP BY hadm_id ) dia ON a.hadm_id = dia.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.drgcodes` GROUP BY hadm_id ) drg ON a.hadm_id = drg.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.emar` GROUP BY hadm_id ) ema ON a.hadm_id = ema.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.hcpcsevents` GROUP BY hadm_id ) hcp ON a.hadm_id = hcp.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.labevents` GROUP BY hadm_id ) lab ON a.hadm_id = lab.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.microbiologyevents` GROUP BY hadm_id ) mic ON a.hadm_id = mic.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.pharmacy` GROUP BY hadm_id ) pha ON a.hadm_id = pha.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.poe` GROUP BY hadm_id ) poe ON a.hadm_id = poe.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.prescriptions` GROUP BY hadm_id ) pre ON a.hadm_id = pre.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.procedures_icd` GROUP BY hadm_id ) pro ON a.hadm_id = pro.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.services` GROUP BY hadm_id ) ser ON a.hadm_id = ser.hadm_id
-- below table needs a join to hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.omr` t INNER JOIN `lcp-internal.mimic_hosp.admissions` adm ON t.subject_id = adm.subject_id AND t.chartdate BETWEEN adm.admittime AND adm.dischtime GROUP BY hadm_id ) omr ON a.hadm_id = omr.hadm_id
-- join to icustays and flag those which have an ICU stay
LEFT JOIN ( SELECT DISTINCT hadm_id FROM `lcp-internal.mimic_icu.icustays` ) icu ON a.hadm_id = icu.hadm_id
"""

tbl_count = pd.read_gbq(query, "lcp-internal")
tbl_count.set_index('hadm_id', inplace=True)
# fill empty values with 0 -> no observations found
for col in tbl_count.columns:
    tbl_count[col] = tbl_count[col].fillna(0)
    tbl_count[col] = tbl_count[col].astype(int)

tbl_count.head()

In [ ]:
table_list = tbl_count.columns[1:]
tbl_agg = pd.DataFrame(index=table_list)

tbl_agg['Data completion'] = (tbl_count > 0).sum()
N = tbl_count.shape[0]
tbl_agg['Data completion'] = tbl_agg['Data completion'].apply(lambda x: f'{x:,} ({x/N:3.2%})')

# iterate over 3 quantile values and output to the table
quantiles_cols = [[0.05, '5th percentile'], [0.5, 'Median number of observations'], [0.95, '95th percentile']]
for q, quantile_col_name in quantiles_cols:
    tbl_agg[quantile_col_name] = 0
    for col in table_list:
        idx = tbl_count[col] > 0
        tbl_agg.loc[col, quantile_col_name] = tbl_count.loc[idx, col].quantile(q)


# repeat for ICU stays

idx_icu = tbl_count['has_icu_stay'] == 1

tbl_agg['No data (ICU subset)'] = 0
for col in table_list:
    tbl_agg.loc[col, 'No data (ICU subset)'] = (tbl_count.loc[idx_icu, col] > 0).sum()

N = idx_icu.sum()
tbl_agg['No data (ICU subset)'] = tbl_agg['No data (ICU subset)'].apply(lambda x: f'{x:,} ({x/N:3.2%})')

for q, quantile_col_name in quantiles_cols:
    quantile_col_name += ' (ICU subset)'
    tbl_agg[quantile_col_name] = 0
    for col in table_list:
        idx = (tbl_count[col] > 0) & (idx_icu)
        tbl_agg.loc[col, quantile_col_name] = tbl_count.loc[idx, col].quantile(q)

tbl_agg.index = [c[2:] for c in tbl_agg.index]
tbl_agg

In [ ]:
import seaborn as sns

# convert tbl_count to long-form
table_list = tbl_count.columns[1:]
tbl_obs_long = pd.DataFrame(columns=['hadm_id', 'count', 'table'])
for table in table_list:
    tmp_df = tbl_count[table].reset_index().copy()
    tmp_df.columns = ['hadm_id', 'count']
    tmp_df['table'] = table[2:]
    
    tbl_obs_long = pd.concat([tbl_obs_long, tmp_df], axis=0)
tbl_obs_long['count'] = tbl_obs_long['count'].astype(int)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=[12, 10])
sns.violinplot(y='count', x='table', data=tbl_obs_long)

In [ ]:
# Figure 3: Data completion grouped by table.

query = """
SELECT a.hadm_id
,tra.n AS n_transfers
,dia.n AS n_diagnoses_icd
,drg.n AS n_drgcodes
,ema.n AS n_emar
,hcp.n AS n_hcpcsevents
,lab.n AS n_labevents
,mic.n AS n_microbiologyevents
,omr.n AS n_omr
,pha.n AS n_pharmacy
,poe.n AS n_poe
,pre.n AS n_prescriptions
,pro.n AS n_procedures_icd
,ser.n AS n_services
FROM `lcp-internal.mimic_icu.icustays` a
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.transfers` GROUP BY hadm_id ) tra ON a.hadm_id = tra.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.diagnoses_icd` GROUP BY hadm_id ) dia ON a.hadm_id = dia.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.drgcodes` GROUP BY hadm_id ) drg ON a.hadm_id = drg.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.emar` GROUP BY hadm_id ) ema ON a.hadm_id = ema.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.hcpcsevents` GROUP BY hadm_id ) hcp ON a.hadm_id = hcp.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.labevents` GROUP BY hadm_id ) lab ON a.hadm_id = lab.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.microbiologyevents` GROUP BY hadm_id ) mic ON a.hadm_id = mic.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.pharmacy` GROUP BY hadm_id ) pha ON a.hadm_id = pha.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.poe` GROUP BY hadm_id ) poe ON a.hadm_id = poe.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.prescriptions` GROUP BY hadm_id ) pre ON a.hadm_id = pre.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.procedures_icd` GROUP BY hadm_id ) pro ON a.hadm_id = pro.hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.services` GROUP BY hadm_id ) ser ON a.hadm_id = ser.hadm_id
-- below table needs a join to hadm_id
LEFT JOIN ( SELECT hadm_id, COUNT(*) AS N FROM `lcp-internal.mimic_hosp.omr` t INNER JOIN `lcp-internal.mimic_hosp.admissions` adm ON t.subject_id = adm.subject_id AND t.chartdate BETWEEN adm.admittime AND adm.dischtime GROUP BY hadm_id ) omr ON a.hadm_id = omr.hadm_id
"""

tbl_count = pd.read_gbq(query)
tbl_count.set_index('hadm_id', inplace=True)
# fill empty values with 0 -> no observations found
for col in tbl_count.columns:
    tbl_count[col] = tbl_count[col].fillna(0)
    tbl_count[col] = tbl_count[col].astype(int)

tbl_count.head()

In [ ]:
tbl_agg = pd.DataFrame(index=tbl_count.columns)

tbl_agg['No data'] = (tbl_count == 0).sum()

# iterate over 3 quantile values and output to the table
quantiles_cols = [[0.05, '5th percentile'], [0.5, 'Median number of observations'], [0.95, '95th percentile']]
for q, quantile_col_name in quantiles_cols:
    tbl_agg[quantile_col_name] = 0
    for col in tbl_count.columns:
        idx = tbl_count[col] > 0
        tbl_agg.loc[col, quantile_col_name] = tbl_count.loc[idx, col].quantile(q)
tbl_agg.index = [c[2:] for c in tbl_agg.index]
tbl_agg